In [9]:
from IGraph import *
from AbstractGraph import *
from UGraphModel import *
from DGraphModel import *
from BayesNet import *

In [16]:
model = DGraphModel()
for idx in range(10):
    model.add(str(idx))

model.connect('0', '1')
model.connect('0', '5')
model.connect('1', '7')
model.connect('3', '2')
model.connect('3', '4')
model.connect('3', '7')
model.connect('3', '8')
model.connect('4', '8')
model.connect('6', '0')
model.connect('6', '1')
model.connect('6', '2')
model.connect('8', '2')
model.connect('8', '7')
model.connect('9', '4')
model.println()

Vertices:
 V(0, in:1, out:2)
 V(1, in:2, out:1)
 V(2, in:3, out:0)
 V(3, in:0, out:4)
 V(4, in:2, out:1)
 V(5, in:1, out:0)
 V(6, in:0, out:3)
 V(7, in:3, out:0)
 V(8, in:2, out:2)
 V(9, in:0, out:1)
------------------------------------------------------
Edges:
 E(0, 1, 0)
 E(0, 5, 0)
 E(1, 7, 0)
 E(3, 2, 0)
 E(3, 4, 0)
 E(3, 7, 0)
 E(3, 8, 0)
 E(4, 8, 0)
 E(6, 0, 0)
 E(6, 1, 0)
 E(6, 2, 0)
 E(8, 2, 0)
 E(8, 7, 0)
 E(9, 4, 0)



In [24]:
DFS = 0
BFS = 1
sorter = TopoSorter(model)
topo = sorter.sort(BFS)
print("topoBFS:", topo)

topo = sorter.sort(DFS)
print("topoDFS:", topo)

topoBFS: ['3', '6', '9', '0', '4', '1', '5', '8', '2', '7']
topoDFS: ['3', '6', '0', '5', '1', '9', '4', '8', '2', '7']


In [7]:
a = np.array([[ 'True', 'True', 'True', 1.0 ],
             [ 'True', 'True', 'False', 0.0 ],
             [ 'True', 'False', 'True', 1.0 ],
             [ 'True', 'False', 'False', 0.0 ],
             [ 'False', 'True', 'True', 1.0 ],
             [ 'False', 'True', 'False', 0.0 ],
             [ 'False', 'False', 'True', 0.0 ],
             [ 'False', 'False', 'False', 1.0 ]])

In [8]:
import numpy as np

In [33]:
np.where( ( a[0] == 'True' ) & ( a[1] == 'True' ) )

(array([0, 1]),)

In [26]:
np.where( a[1] == 'True' )

(array([0, 1, 4, 5]),)

In [72]:
a[1:,[0, 1, 2, 3]]

array([['True', 'True', 'False', 'False'],
       ['True', 'False', 'True', 'False'],
       ['1.0', '0.0', '1.0', '0.0']], dtype='<U5')

In [1261]:
class ConditionalProbabilityTable():
    def __init__(self,initTable,initParents,initNodeName):
        self.ProbabilityTable = np.array(initTable).T.astype(dtype='<U21')  
        self.ParentsList = initParents
        self.NodeName = initNodeName
        if self.isDiscreteDistribution() == False:
            self.ParentsNum = len(initParents)
            if self.ProbabilityTable.shape[0]-2 != self.ParentsNum:
                raise NameError("Init failed, number of parents is wrong!")
        
    def getValueList(self):
        return list(set(self.ProbabilityTable[-2]))

    def isDiscreteDistribution(self):
        if self.ParentsList == None:
            return True
        else:
            False
    
    def getNodeProbability(self,ParentState):
        RunTable = self.ProbabilityTable
        for name in self.ParentsList:
            if name in ParentState:
                # Check Parent len
                State = np.array(ParentState[name]).astype(dtype='<U21')
                index = np.where( RunTable[0] == State )[0]
                RunTable = RunTable[1:,index]  
            else:
                raise NameError("Parents state is not enough")
        return RunTable
    
    def getOutput(self,ParentState):
        if self.isDiscreteDistribution():   
            sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
        else:
            NodeProba = self.getNodeProbability(ParentState)
            sortedProba = NodeProba.T[NodeProba[1].argsort()].T
            
        RandomValue = random.random()
        #print(RandomValue)
        #print(sortedProba)
        for index in range(len(sortedProba)):
            if RandomValue < sortedProba[1][:index+1].astype(np.float).sum():
                ParentState[self.NodeName] = sortedProba[0,index]
                return ParentState
        ParentState[self.NodeName] = sortedProba[0,-1]
        return ParentState

    def getNodeProbabilityWithLikehood(self, ParentState, ParentLikelihoodWeight):
        RunTable = self.ProbabilityTable
        Likelihood = 1
        for name in self.ParentsList:
            if name in ParentState:
                # Check Parent len
                Likelihood *= ParentLikelihoodWeight[name]
                State = np.array(ParentState[name]).astype(dtype='<U21')
                index = np.where( RunTable[0] == State )[0]
                RunTable = RunTable[1:,index]  
            else:
                raise NameError("Parents state is not enough")
        return RunTable,Likelihood
        
    def getOutputWithLikehood(self, ParentState, ParentLikelihoodWeight, Observations):
        if self.NodeName not in Observations.keys():
            if self.isDiscreteDistribution():   
                sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
            else:
                NodeProba,Likelihood = self.getNodeProbabilityWithLikehood(ParentState,ParentLikelihoodWeight)
                sortedProba = NodeProba.T[NodeProba[1].argsort()].T

            RandomValue = random.random()
            #print(RandomValue)
            #print(sortedProba)
            for index in range(len(sortedProba)):
                if RandomValue < sortedProba[1][:index+1].astype(np.float).sum():
                    ParentState[self.NodeName] = sortedProba[0,index]
                    if self.isDiscreteDistribution(): 
                        ParentLikelihoodWeight[self.NodeName] = float(sortedProba[1,index])  
                    else:
                        ParentLikelihoodWeight[self.NodeName] = Likelihood
                    return ParentState, ParentLikelihoodWeight
            ParentState[self.NodeName] = sortedProba[0,-1]
            if self.isDiscreteDistribution(): 
                ParentLikelihoodWeight[self.NodeName] = float(sortedProba[1,index])
            else:
                ParentLikelihoodWeight[self.NodeName] = Likelihood
            return ParentState, ParentLikelihoodWeight
        else:
            if self.isDiscreteDistribution():   
                sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
            else:
                NodeProba,Likelihood = self.getNodeProbabilityWithLikehood(ParentState,ParentLikelihoodWeight)
                sortedProba = NodeProba.T[NodeProba[1].argsort()].T
            #print(sortedProba)
            #print(np.array(Observations[self.NodeName]).astype(dtype='<U21'))
            index = np.where(sortedProba[0]==np.array(Observations[self.NodeName]).astype(dtype='<U21'))[0][0]
            #print("=============",index)
            if self.isDiscreteDistribution(): 
                ParentLikelihoodWeight[self.NodeName] = float(sortedProba[1,index])
            else:
                ParentLikelihoodWeight[self.NodeName] = Likelihood*float(sortedProba[1,index])
                
            ParentState[self.NodeName] = sortedProba[0,index]
            
            return ParentState, ParentLikelihoodWeight

In [1262]:
asia = ConditionalProbabilityTable( [ ['True', 0.5],[ 'False', 0.5 ]], None, 'asia' )

In [1263]:
tuberculosis = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.2 ],
     [ 'True', 'False', 0.8 ],
     [ 'False', 'True', 0.01 ],
     [ 'False', 'False', 0.99 ]], ['asia'],'tuberculosis')

In [1264]:
smoking = ConditionalProbabilityTable( [ ['True', 0.5],[ 'False', 0.5 ]], None, 'smoking' )

In [1265]:
lung = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.75 ],
     [ 'True', 'False', 0.25 ],
     [ 'False', 'True', 0.02 ],
     [ 'False', 'False', 0.98 ]], ['smoking'], 'lung' )

bronchitis = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.92 ],
     [ 'True', 'False', 0.08 ],
     [ 'False', 'True', 0.03 ],
     [ 'False', 'False', 0.97 ]], ['smoking'], 'bronchitis' )

tuberculosis_or_cancer = ConditionalProbabilityTable(
    [[ 'True', 'True', 'True', 1.0 ],
     [ 'True', 'True', 'False', 0.0 ],
     [ 'True', 'False', 'True', 1.0 ],
     [ 'True', 'False', 'False', 0.0 ],
     [ 'False', 'True', 'True', 1.0 ],
     [ 'False', 'True', 'False', 0.0 ],
     [ 'False', 'False', 'True', 0.0 ],
     [ 'False', 'False', 'False', 1.0 ]], ['tuberculosis', 'lung'], 'tuberculosis_or_cancer' )

In [1266]:
xray = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.885 ],
     [ 'True', 'False', 0.115 ],
     [ 'False', 'True', 0.04 ],
     [ 'False', 'False', 0.96 ]], ['tuberculosis_or_cancer'] , 'xray')

In [1267]:
dyspnea = ConditionalProbabilityTable(
    [[ 'True', 'True', 'True', 0.96 ],
     [ 'True', 'True', 'False', 0.04 ],
     [ 'True', 'False', 'True', 0.89 ],
     [ 'True', 'False', 'False', 0.11 ],
     [ 'False', 'True', 'True', 0.96 ],
     [ 'False', 'True', 'False', 0.04 ],
     [ 'False', 'False', 'True', 0.89 ],
     [ 'False', 'False', 'False', 0.11 ]], ['tuberculosis_or_cancer', 'bronchitis'],'dyspnea')

In [1101]:
NodeList = [asia,smoking,tuberculosis,lung,tuberculosis_or_cancer,bronchitis,xray,dyspnea]

In [1145]:
RunningVar

{'asia': 'True',
 'smoking': 'True',
 'tuberculosis': 'False',
 'lung': 'False',
 'tuberculosis_or_cancer': 'False',
 'bronchitis': 'True',
 'xray': 'False',
 'dyspnea': 'True'}

In [1134]:
RunForwardList = []
for i in range(300000):
    RunningVar = {}
    RunningVar = asia.getOutput(RunningVar)
    RunningVar = smoking.getOutput(RunningVar)
    RunningVar = tuberculosis.getOutput(RunningVar)
    RunningVar = lung.getOutput(RunningVar)
    RunningVar = tuberculosis_or_cancer.getOutput(RunningVar)
    RunningVar = bronchitis.getOutput(RunningVar)
    RunningVar = xray.getOutput(RunningVar)
    RunningVar = dyspnea.getOutput(RunningVar)
    RunForwardList.append(RunningVar)

In [1135]:
Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }

In [1136]:
NodeNameList = ['asia','smoking','tuberculosis','lung','tuberculosis_or_cancer','bronchitis','xray','dyspnea']

In [1137]:
RunValueList = []
KeyList = RunForwardList[0].keys()
for RunForwardValue in RunForwardList:
    OneRow = []
    ObservationsCheck = True
    for key in Observations:
        if Observations[key] != RunForwardValue[key]:
            ObservationsCheck = False
    if ObservationsCheck == True:
        for key in RunForwardValue: 
            OneRow.append(RunForwardValue[key])
        RunValueList.append(OneRow)
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))

In [1138]:
NumpyKey = np.array(list(KeyList))

In [1139]:
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,len(np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0])/NumpyRunValue.shape[0])
    print('============================')

asia
True 0.9562624254473161
False 0.0437375745526839
smoking False
tuberculosis True
lung
True 0.02385685884691849
False 0.9761431411530815
tuberculosis_or_cancer
True 1.0
False 0.0
bronchitis True
xray
True 0.8846918489065606
False 0.11530815109343936
dyspnea
True 0.9602385685884692
False 0.039761431411530816


In [1155]:
# Likelihood

In [ ]:
RunForwardList = []
for i in range(300000):
    RunningVar = {}
    ParentLikelihoodWeight = {}
    Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }
    RunningVar = asia.getOutput(RunningVar,ParentLikelihoodWeight)
    RunningVar = smoking.getOutput(RunningVar)
    RunningVar = tuberculosis.getOutput(RunningVar)
    RunningVar = lung.getOutput(RunningVar)
    RunningVar = tuberculosis_or_cancer.getOutput(RunningVar)
    RunningVar = bronchitis.getOutput(RunningVar)
    RunningVar = xray.getOutput(RunningVar)
    RunningVar = dyspnea.getOutput(RunningVar)
    RunForwardList.append(RunningVar)

In [1317]:
RunningVar = {}
ParentLikelihoodWeight = {}
Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }
RunningVar,ParentLikelihoodWeight = asia.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
RunningVar,ParentLikelihoodWeight = smoking.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
RunningVar,ParentLikelihoodWeight = tuberculosis.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
RunningVar,ParentLikelihoodWeight = lung.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
RunningVar,ParentLikelihoodWeight = tuberculosis_or_cancer.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
RunningVar,ParentLikelihoodWeight = bronchitis.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
RunningVar,ParentLikelihoodWeight = xray.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
RunningVar,ParentLikelihoodWeight = dyspnea.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)

In [1318]:
RunningVar,ParentLikelihoodWeight

({'asia': 'False',
  'smoking': 'False',
  'tuberculosis': 'True',
  'lung': 'False',
  'tuberculosis_or_cancer': 'True',
  'bronchitis': 'True',
  'xray': 'True',
  'dyspnea': 'False'},
 {'asia': 0.5,
  'smoking': 0.5,
  'tuberculosis': 0.005,
  'lung': 0.5,
  'tuberculosis_or_cancer': 0.0025,
  'bronchitis': 0.015,
  'xray': 0.0025,
  'dyspnea': 3.75e-05})

In [930]:
running_list = []

In [944]:
import pandas as pd

In [1255]:
test = ConditionalProbabilityTable(
    [[ 1, True, 'A', 0.3 ],
     [ 1, True, 'B', 0.4 ],
     [ 1, True, 'C', 0.3 ],
     [ 1, False, 'A', 0.2 ],
     [ 1, False, 'B', 0.4 ],
     [ 1, False, 'C', 0.4 ],
     [ 2, True, 'A', 0.1 ],
     [ 2, True, 'B', 0.9 ],
     [ 2, True, 'C', 0.0 ],
     [ 2, False, 'A', 0.0 ],
     [ 2, False, 'B', 0.4 ],
     [ 2, False, 'C', 0.6]], ['khang', 'chau'], 'tretrau' )

In [1199]:
a = test.getOutput({'khang': 1, 'khi':'D' , 'chau':False, 'cho': 'C'})

In [1246]:
list(set(test.ProbabilityTable[-2]))

['2.0', '1.0', '3.0']

In [1259]:
test = ConditionalProbabilityTable(   
    [[1, 0.333 ],
     [2, 0.4 ],
     [3, 0.3 ],], None, 'heo' )

In [1260]:
b = test.getOutputWithLikehood({'khang': 1, 'khi':'D' , 'chau':False, 'cho': 'C'},{},{'heo':'1.0'})

[['3.0' '1.0' '2.0']
 ['0.3' '0.333' '0.4']]
1.0
============= 1


In [1249]:
b[0]

{'khang': 1, 'khi': 'D', 'chau': False, 'cho': 'C', 'heo': '1.0'}

In [1254]:
np.array([1.2,2]).astype(dtype='<U21')

array(['1.2', '2.0'], dtype='<U21')

In [1223]:
l = np.array([[ 'True', 'True', 0.2 ],
     [ 'True', 'False', 0.8 ],
     [ 'False', 'True', 0.01 ],
     [ 'False', 'False', 0.99 ]]).T

In [1227]:
l[np.where( l[0] == 'True'  )[0]]

array([0, 1])

In [1231]:
l[:,np.where( l[0] == 'True'  )[0]]

array([['True', 'True'],
       ['True', 'False'],
       ['0.2', '0.8']], dtype='<U5')

In [1236]:
l[1:,1:]

array([['False', 'True', 'False'],
       ['0.8', '0.01', '0.99']], dtype='<U5')

In [509]:
k = np.array([[ 'True', 'True', 'True', 0.96 ],
     [ 'True', 'True', 'False', 0.04 ],
     [ 'True', 'False', 'True', 0.89 ],
     [ 'True', 'False', 'False', 0.11 ],
     [ 'False', 'True', 'True', 0.96 ],
     [ 'False', 'True', 'False', 0.04 ],
     [ 'False', 'False', 'True', 0.89 ],
     [ 'False', 'False', 'False', 0.11 ]])

In [655]:
np.array(1).astype(dtype='<U21')

array('1', dtype='<U21')

In [386]:
k['s'] =1

In [387]:
's' in k

True

In [ ]:
list(map(list, zip(*l)))

In [584]:
l[4:]

[[1, 4, 2, 0.4],
 [1, 4, 3, 0.4],
 [2, 3, 1, 0.1],
 [2, 3, 2, 0.9],
 [2, 3, 3, 0.0],
 [2, 4, 1, 0.0],
 [2, 4, 2, 0.4],
 [2, 4, 3, 0.6]]